In [ ]:
"""
Get data of movie adapatations from IMDb page. Advanced search "based on novel":
https://www.imdb.com/search/title/?title_type=feature\
&release_date=1930-01-01,2020-12-31\
&countries=us&keywords=based+on+novel\
&languages=en&count=100

Scrape 20 pages of search results for movie title and link to individual movie page.
Scrape 2000 movie pages.

functions used:
get_soup(url) from scrape.py
get_movie_detail(movie_soup) from get_movie.py

Output to pickle 'movie_adaptations_data'.

"""

### Construct DataFrame of movie adaptations

In [6]:
from bs4 import BeautifulSoup
import requests

import pandas as pd
import numpy as np

import pickle

import sys
sys.path.append('/Users/katiehuang/Documents/metis/projects/onl_ds5_project_2/py')
from scrape import *
from get_movie import *

In [16]:
%run -i '../py/get_movie.py'

In [2]:
# Find url for 20 pages: url_list

base_url="https://www.imdb.com/search/title/?title_type=feature\
&release_date=1930-01-01,2020-12-31\
&countries=us&keywords=based+on+novel\
&languages=en&count=100"

url_list=[base_url]
for i in range(101,2001,100):
    url = base_url+ "&start=" + str(i) + "&ref_=adv_nxt"
    url_list.append(url)

In [9]:
# Create soup_list (20 soup for 20 pages)

soup_list=[]
for url in url_list:
    soup_list.append(get_soup(url))

In [11]:
# Create movie_url_list (individual webpage for each movie)

movie_url_list=[]
movie_base_url = "https://www.imdb.com/"

for soup in soup_list:
    divs= [div for div in soup.find_all('div', class_="lister-item-content")]

    for i in range(100):
        movie_url = divs[i].find('a').get('href')
        movie_url_list.append(movie_base_url + movie_url)

In [13]:
# Create movie_soup_list for the movies: len=2000

movie_soup_list=[]
for movie_url in movie_url_list:
    movie_soup = get_soup(movie_url)
    movie_soup_list.append(movie_soup)

In [18]:
# Create list of movie_dict: len=2000
movie_detail_list=[]
for movie_soup in movie_soup_list:
    movie = get_movie_detail(movie_soup)
    movie_detail_list.append(movie)

In [21]:
# Create pd DataFrame of 2000 movies!
movie_df = pd.DataFrame(movie_detail_list)
movie_df.sort_values('gross_world',ascending=False).head(3)

,movie_title,rating,vote,certificate,genre,release_date,metascore,keywords,budget,opening_weekend_usa,...,runtime,distributor,language,country,director,writer,star,link_d,link_w,link_s
22,The Lord of the Rings: The Return of the King,8.9,1639885,PG-13,"[Action, Adventure, Drama]",2003-12-17,94.0,"[epic, orc, hobbit, ring, battle]",94000000.0,72629713.0,...,192.0,NewLineCinema,"[English, Quenya, OldEnglish, Sindarin]","[NewZealand, USA]",Peter Jackson,"[J.R.R. Tolkien, Fran Walsh, 2 more credits]","[Elijah Wood, Viggo Mortensen, Ian McKellen]",/name/nm0001392/,"[/name/nm0866058/, /name/nm0909638/, fullcredi...","[/name/nm0000704/, /name/nm0001557/, /name/nm0..."
179,Pirates of the Caribbean: On Stranger Tides,6.6,480574,PG-13,"[Action, Adventure, Fantasy]",2011-05-20,NaN,"[pirate, captain, blackbeard, soldier, revenge]",250000000.0,90151958.0,...,137.0,WaltDisneyPictures,"[English, Spanish]","[USA, UK]",Rob Marshall,"[Ted Elliott, Terry Rossio, 7 more credits]","[Johnny Depp, Penélope Cruz, Ian McShane]",/name/nm0551128/,"[/name/nm0254645/, /name/nm0744429/, fullcredi...","[/name/nm0000136/, /name/nm0004851/, /name/nm0..."
66,Jurassic Park,8.1,866180,PG-13,"[Action, Adventure, Sci-Fi]",1993-06-11,68.0,"[dinosaur, tyrannosaurus rex, sneeze, severed ...",63000000.0,47026828.0,...,127.0,UniversalPictures,"[English, Spanish]",[USA],Steven Spielberg,"[Michael Crichton, Michael Crichton]","[Sam Neill, Laura Dern, Jeff Goldblum]",/name/nm0000229/,"[/name/nm0000341/, /name/nm0000341/]","[/name/nm0000554/, /name/nm0000368/, /name/nm0..."


In [22]:
# Save the data to pickle file
movie_df.to_pickle('../data/movie_data_adaptation')